# Importing libraries

In [7]:
import keras
import numpy as np
import librosa
import warnings
import joblib
warnings.filterwarnings("ignore")
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

## creating a Class to return predictions on emotions

In [21]:
class livePredictions:
    """
    Main class of the application.
    """

    def __init__(self, path, file):
        """
        Init method is used to initialize the main parameters.
        """
        self.path = path
        self.file = file

    def load_model(self):
        """
        Method to load the chosen model.
        :param path: path to your h5 model.
        :return: summary of the model with the .summary() function.
        """
        self.loaded_model = keras.models.load_model(self.path)
        print("*"*30)
        print("loading the model")
        print("*"*30)
#         return self.loaded_model.summary()
    
    def load_audio_file(self):
        """
        Method to load an audio file with given file path
        """
        data, sampling_rate = librosa.load(self.file)
        print("*"*30)
        print("loading the audio file")
        print("*"*30)
        return data, sampling_rate  
    
    def audio_pre_process(self):
        """
        Method to perform feature extraction on an audio file
        """
        temp = []
        Signal, sampling_rate = self.load_audio_file()
        energy = librosa.feature.rms(y=Signal)
        temp.append(energy.mean())

        zcr = librosa.feature.zero_crossing_rate(Signal)
        temp.append(zcr.mean())

        S = np.abs(librosa.stft(Signal))
        pitches, magnitudes = librosa.piptrack(S=S)
        temp.append(pitches.mean())

        mfccs = librosa.feature.mfcc(y=Signal)
        temp.extend(list(mfccs.mean(axis=1)))
        temp = np.array(temp)
        scale = joblib.load("scaling_object.sav")
        temp = scale.transform(np.array(temp).reshape(1, -1))
        x = np.reshape(temp,(1,temp.shape[1],1))
        print("*"*30)
        print("Extracting the features from the Audio file")
        print("*"*30)
        return x

    def makepredictions(self):
        """
        Method to process the files and create your features.
        """
        x = self.audio_pre_process()
        predictions = self.loaded_model.predict(x)
        print("*"*30)
        print("Prediction is: ", self.convertclasstoemotion(predictions))
        print("*"*30)

    @staticmethod
    def convertclasstoemotion(pred):
        """
        Method to convert the predictions (int) into human readable strings.
        """
        
        label_conversion = {'0': 'neutral',
                            '1': 'calm',
                            '2': 'happy',
                            '3': 'sad',
                            '4': 'angry',
                            '5': 'fearful',
                            '6': 'disgust',
                            '7': 'surprised'}

        for key, value in label_conversion.items():
            if int(key) == np.argmax(pred,axis=-1):
                label = value
        return label


In [22]:
pred = livePredictions(path='saved_model_60-40.hdf5',file=r'TESS/OAF_back_angry.wav')

pred.load_model()
pred.makepredictions()

******************************
loading the model
******************************
******************************
loading the audio file
******************************
******************************
Extracting the features from the Audio file
******************************
******************************
Prediction is:  angry
******************************
